# Visulization of Built-In Interval Estimations on a 1d Signal

This notebook provide an example of our built-in interval estimations: We give such estimations on a 1d signal:
$$
f(X)=\sin(2\pi X)+0.5X^2, X\sim Unif[0,1]
$$

## Helper Cells

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("../src"))
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy.stats import norm
from tqdm import tqdm
from itertools import product
from sklearn.model_selection import ParameterSampler
from BRAT.algorithms import BRATD
from BRAT.utils import generate_data, find_min_scale

In [2]:
plt.style.use('../matplotlibrc')
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6

In [3]:
import matplotlib.colors as mcolors

def darken(color, factor=0.7):
    """
    Darken a hex color by multiplying its RGB components by `factor` (0 < factor < 1).
    """
    rgb = np.array(mcolors.to_rgb(color))
    darker = np.clip(rgb * factor, 0, 1)
    return mcolors.to_hex(darker)

The following function helps us calibrate the intervals to increase coverage.

In [4]:
def calibrate_scale(widths_cal, y_cal, y_pred_cal, alpha=0.05):
    """
    Two-stage search for global c so that empirical coverage ≈ 1-alpha.
    """
    n = len(y_cal)
    def cov(c):
        lo = y_pred_cal - c * widths_cal
        hi = y_pred_cal + c * widths_cal
        return np.mean((y_cal >= lo) & (y_cal <= hi))

    # exponential search
    c = 1.0
    if cov(c) < 1 - alpha:
        while cov(c) < 1 - alpha:
            c *= 2
    C = c

    # binary refine on [0, C]
    lo, hi = 0.0, C
    tol = 2.0 / n
    while hi - lo > tol:
        mid = (lo + hi) / 2
        if cov(mid) < 1 - alpha:
            lo = mid
        else:
            hi = mid
    return hi

## Visualization

### Data Generation

In [6]:
# data configuration
np.random.seed(42)
n_train = 1000
n_test = 300
n_cal = 200
noise_std = 1

# underlying signal
f = lambda x: np.sin(2*np.pi*x) + 0.5 * x**2

# uniform samples on [0,1]
X_train = np.random.rand(n_train,1)   # shape (n_train,1)
X_test  = np.random.rand(n_test,1)   # shape (n_test,1)
X_cal   = np.random.rand(n_cal,1)   # shape (n_cal,1)
X_plot  = np.linspace(0, 1, n_test).reshape(-1,1)

# true signal
y_train_true = f(X_train).flatten()
y_test_true  = f(X_test).flatten()
y_cal_true   = f(X_cal).flatten()

# simulated observations
y_train = y_train_true + noise_std * np.random.randn(n_train)
y_test  = y_test_true  + noise_std * np.random.randn(n_test)
y_cal   = y_cal_true + noise_std * np.random.randn(n_cal)

### Interval Estimations

In [30]:
# Simply add the parameter of interest here in the dictionary
param_dist = {
    'n_estimators': [50],
    'learning_rate': [0.6],
    'max_depth':      [8],
    'subsample_rate': [1.0],
    'dropout_rate':   [0.95]
}
# If you are interested in one particular set of hps, set n_iter = 1;
# If you would like to explore some combinations of those hps, increase your budget n_iter
n_iter = 1
sampler = list(ParameterSampler(param_dist, n_iter=n_iter, random_state=42))
output_dir = '../experiments/1d/data/'
os.makedirs(output_dir, exist_ok=True)

In [31]:
results = []
# number of repetitions you want to iterate over to get the
# average intervals
reps = 50

for params in tqdm(sampler,
                               desc='Grid Searching...'):
    # extract and cast to the right types
    ne = int(params['n_estimators'])
    lr = float(params['learning_rate'])
    md = int(params['max_depth'])
    sr = float(params['subsample_rate'])
    dr = float(params['dropout_rate'])

    model_results = []
    for model_id in range(reps):
        # Train BRATD model
        bratd = BRATD(n_estimators=ne,
                    learning_rate=lr,
                    max_depth=md,
                    subsample_rate=sr,
                    dropout_rate=dr,
                    disable_tqdm=True)
        bratd.fit(X_train, y_train, X_test, y_test)

        # Predict and compute CIs for each test point
        bratd_pred = []
        bratd_ci_lb = []
        bratd_ci_ub = []
        bratd_pi_lb = []
        bratd_pi_ub = []
        bratd_ri_lb = []
        bratd_ri_ub = []
        counter = 0

        # Precompute the K matrix before entering the test point loop.
        # For each model, all test points share a K matrix
        sigma_hat2 = bratd.est_sigma_hat2(in_bag=False)
        sigma_hat = np.sqrt(sigma_hat2)
        bratd.unif_nystrom(Nystrom_subsample=0.05)
        bratd.sketch_K()
        lam = bratd.learning_rate
        q = 1 - bratd.dropout_rate
        s = (1 + lam * q) / lam

        y_pred_cal = bratd.predict(X_cal)
        res_cal = np.abs(y_pred_cal - y_cal)
        qtel_hat = np.quantile(res_cal, 0.95)

        y_pred_cal = []
        tau_cal    = []
        pi_sigma_cal = []
        for x_cal in X_cal:
            y_pc = bratd.predict(x_cal.reshape(1, -1)).item()
            rn_c = bratd.sketch_r(x_cal.reshape(1, -1), vector=False).item()
            t_c  = s * rn_c * sigma_hat
            p_c  = np.sqrt(sigma_hat2 + t_c**2)

            y_pred_cal.append(y_pc)
            tau_cal.append(t_c)
            pi_sigma_cal.append(p_c)
        pi_sigma_cal = np.array(pi_sigma_cal)

        # calibrate the prediction intervals here
        c = find_min_scale(
            y_true=y_cal,            
            y_pred=y_pred_cal,
            widths=norm.ppf(0.975)*pi_sigma_cal,
            target=0.95
        )

        for i, x in tqdm(enumerate(X_test), desc='Iterating test points...'):
            pred = bratd.predict(x.reshape(1, -1))
            bratd_pred.append(pred)
            rn_norm = bratd.sketch_r(x)
            s = (1 + bratd.learning_rate * (1 - bratd.dropout_rate)) / bratd.learning_rate
            tau_hat2 = (s**2 * rn_norm**2 * sigma_hat2)

            # calibrated confidence interval, prediction interval and reproduction interval
            ci_lb = pred - norm.ppf(0.975) * np.sqrt(tau_hat2)
            ci_ub = pred + norm.ppf(0.975) * np.sqrt(tau_hat2)
            c_ci_lb = pred - c * norm.ppf(0.975) * np.sqrt(tau_hat2)
            c_ci_ub = pred + c * norm.ppf(0.975) * np.sqrt(tau_hat2)

            pi_lb = pred - c * norm.ppf(0.975) * np.sqrt(tau_hat2 + sigma_hat2)
            pi_ub = pred + c * norm.ppf(0.975) * np.sqrt(tau_hat2 + sigma_hat2)
            c_pi_lb = pred - c * norm.ppf(0.975) * np.sqrt(tau_hat2 + sigma_hat2)
            c_pi_ub = pred + c * norm.ppf(0.975) * np.sqrt(tau_hat2 + sigma_hat2)

            ri_lb = pred - c * norm.ppf(0.975) * np.sqrt(2 * tau_hat2)
            ri_ub = pred + c * norm.ppf(0.975) * np.sqrt(2 * tau_hat2)
            c_ri_lb = pred - c * norm.ppf(0.975) * np.sqrt(2 * tau_hat2)
            c_ri_ub = pred + c * norm.ppf(0.975) * np.sqrt(2 * tau_hat2)

            cfml_lower = pred - qtel_hat
            cfml_upper = pred + qtel_hat

            ci_covered = int(ci_lb.item() <= y_test_true[i].item() <= ci_ub.item())
            c_ci_covered = int(c_ci_lb.item() <= y_test_true[i].item() <= c_ci_ub.item())

            cfml_covered = int(cfml_lower.item() <= y_test[i].item() <= cfml_upper.item())

            pi_covered = int(pi_lb.item() <= y_test[i].item() <= pi_ub.item())
            c_pi_covered = int(c_pi_lb.item() <= y_test[i].item() <= c_pi_ub.item())
            
            # store the data
            model_results.append({
                'pt_idx': i,
                'model_id': model_id,
                'pred': pred.item(),
                'obs': y_test[i].item(),
                'truth': y_test_true[i],
                'bias': pred.item() - y_test_true[i],
                'tau_hat2': tau_hat2.item(),
                'sigma_hat2': sigma_hat2.item(),
                'rn_norm': rn_norm,
                'ci_lb': ci_lb.item(),
                'ci_ub': ci_ub.item(),
                'ci_width': ci_ub.item() - ci_lb.item(),
                'ci_covered': ci_covered,
                'c_ci_lb': c_ci_lb.item(),
                'c_ci_ub': c_ci_ub.item(),
                'c_ci_width': c_ci_ub.item() - c_ci_lb.item(),
                'c_ci_covered': c_ci_covered,
                'pi_lb': pi_lb.item(),
                'pi_ub': pi_ub.item(),
                'pi_width': pi_ub.item() - pi_lb.item(),
                'pi_covered': pi_covered,
                'c_pi_lb': c_pi_lb.item(),
                'c_pi_ub': c_pi_ub.item(),
                'c_pi_width': c_pi_ub.item() - c_pi_lb.item(),
                'c_pi_covered': c_pi_covered,
                'ri_lb': ri_lb.item(),
                'ri_ub': ri_ub.item(),
                'ri_width': ri_ub.item() - ri_lb.item(),
                'c_ri_lb': c_ri_lb.item(),
                'c_ri_ub': c_ri_ub.item(),
                'c_ri_width': c_ri_ub.item() - c_ri_lb.item(),
                'cfml_lower': cfml_lower.item(),
                'cfml_upper': cfml_upper.item(),
                'cfml_width': cfml_upper.item() - cfml_lower.item(),
                'cfml_covered': cfml_covered,
            })
    
    df = pd.DataFrame(model_results)
    df['ci_mean_lb'] = df.groupby('pt_idx')['ci_lb'].transform('mean')
    df['ci_mean_ub'] = df.groupby('pt_idx')['ci_ub'].transform('mean')
    df['c_ci_mean_lb'] = df.groupby('pt_idx')['c_ci_lb'].transform('mean')
    df['c_ci_mean_ub'] = df.groupby('pt_idx')['c_ci_ub'].transform('mean')
    df['pi_mean_lb'] = df.groupby('pt_idx')['pi_lb'].transform('mean')
    df['pi_mean_ub'] = df.groupby('pt_idx')['pi_ub'].transform('mean')
    df['c_pi_mean_lb'] = df.groupby('pt_idx')['c_pi_lb'].transform('mean')
    df['c_pi_mean_ub'] = df.groupby('pt_idx')['c_pi_ub'].transform('mean')
    df['ri_mean_lb'] = df.groupby('pt_idx')['ri_lb'].transform('mean')
    df['ri_mean_ub'] = df.groupby('pt_idx')['ri_ub'].transform('mean')
    df['c_ri_mean_lb'] = df.groupby('pt_idx')['c_ri_lb'].transform('mean')
    df['c_ri_mean_ub'] = df.groupby('pt_idx')['c_ri_ub'].transform('mean')
    df['cfml_mean_lb'] = df.groupby('pt_idx')['cfml_lower'].transform('mean')
    df['cfml_mean_ub'] = df.groupby('pt_idx')['cfml_upper'].transform('mean')

    ci_coverage_rate = df.groupby('pt_idx')['ci_covered'].mean()
    c_ci_coverage_rate = df.groupby('pt_idx')['c_ci_covered'].mean()
    df['ci_coverage'] = ci_coverage_rate
    df['c_ci_coverage'] = c_ci_coverage_rate
    mean_ci_coverage = df['ci_coverage'].mean()
    mean_c_ci_coverage = df['c_ci_coverage'].mean()
    df['mean_ci_coverage'] = mean_ci_coverage
    df['mean_c_ci_coverage'] = mean_c_ci_coverage

    cfml_coverage_rate = df.groupby('pt_idx')['cfml_covered'].mean()
    df['cfml_coverage'] = cfml_coverage_rate
    mean_cfml_coverage = df['cfml_coverage'].mean()
    df['mean_cfml_coverage'] = mean_cfml_coverage

    pi_coverage_rate = df.groupby('pt_idx')['pi_covered'].mean()
    c_pi_coverage_rate = df.groupby('pt_idx')['c_pi_covered'].mean()
    df['pi_coverage'] = pi_coverage_rate
    df['c_pi_coverage'] = c_pi_coverage_rate
    mean_pi_coverage = df['pi_coverage'].mean()
    mean_c_pi_coverage = df['c_pi_coverage'].mean()
    df['mean_pi_coverage'] = mean_pi_coverage
    df['mean_c_pi_coverage'] = mean_c_pi_coverage

    ri_coverage_rate = []
    c_ri_coverage_rate = []
    for _, row in df.iterrows():
        ri_coverage_rate.append(
            ((df['pred'] >= row['ri_lb']) & (df['pred'] <= row['ri_ub'])).mean()
        )
        c_ri_coverage_rate.append(
            ((df['pred'] >= row['c_ri_lb']) & (df['pred'] <= row['c_ri_ub'])).mean()
        )


    df['ri_coverage'] = ri_coverage_rate
    df['c_ri_coverage'] = c_ri_coverage_rate
    mean_ri_coverage = df['ri_coverage'].mean()
    mean_c_ri_coverage = df['c_ri_coverage'].mean()
    df['mean_ri_coverage'] = mean_ri_coverage
    df['mean_c_ri_coverage'] = mean_c_ri_coverage
    results.append(df)

    # data saved to the following directory. Recommend using data wrangler to see the distributional results of them
    file_name = f"ne_{ne}_lr_{lr}_md_{md}_sr_{sr}_dr_{dr}.csv"
    df.to_csv(os.path.join(output_dir, file_name), index=False)

    bratd_mean = df.groupby('pt_idx')['pred'].mean().values
    bratd_mean_ci_lb = df.groupby('pt_idx')['ci_lb'].mean().values
    bratd_mean_ci_ub = df.groupby('pt_idx')['ci_ub'].mean().values
    bratd_mean_pi_lb = df.groupby('pt_idx')['pi_lb'].mean().values
    bratd_mean_pi_ub = df.groupby('pt_idx')['pi_ub'].mean().values
    bratd_mean_ri_lb = df.groupby('pt_idx')['ri_lb'].mean().values
    bratd_mean_ri_ub = df.groupby('pt_idx')['ri_ub'].mean().values
    bratd_mean_cfml_lb = df.groupby('pt_idx')['cfml_lower'].mean().values
    bratd_mean_cfml_ub = df.groupby('pt_idx')['cfml_upper'].mean().values
    
    sort_idx = np.argsort(X_test.ravel())
    X_test_s = X_test.ravel()[sort_idx]
    pred_s = bratd_mean[sort_idx]
    ci_lb_s = bratd_mean_ci_lb[sort_idx]
    ci_ub_s = bratd_mean_ci_ub[sort_idx]
    pi_lb_s = bratd_mean_pi_lb[sort_idx]
    pi_ub_s = bratd_mean_pi_ub[sort_idx]
    ri_lb_s = bratd_mean_ri_lb[sort_idx]
    ri_ub_s = bratd_mean_ri_ub[sort_idx]
    cfml_lb_s = bratd_mean_cfml_lb[sort_idx]
    cfml_ub_s = bratd_mean_cfml_ub[sort_idx]

    data_dir = './1d/data'
    os.makedirs(data_dir, exist_ok=True)

    # -------- Plotting ----------- #
    fig, axes = plt.subplots(1, 4, figsize=(8, 2), sharey=True)

    true_plot = f(X_plot)

    # Confidence Interval
    ax = axes[0]
    ax.scatter(X_train, y_train, color='grey', alpha=0.3, s=5, label='Training Data')
    ax.plot(X_test_s, pred_s, '-', color=darken("#00BEFF",1.0), linewidth=1, label='Mean Prediction')
    ax.fill_between(X_test_s, ci_lb_s, ci_ub_s, facecolor="#00BEFF", edgecolor="#00BEFF", alpha=0.4)
    ax.plot(X_plot, true_plot, 'k--', linewidth=1, zorder=5, label='True Function')  
    ax.text(0.05, 0.95, f'Coverage: {mean_ci_coverage:.2%}',
            transform=ax.transAxes, fontsize=8, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.2))
    leg = ax.legend(['Training Data', 'Prediction', 'CI', 'True Function'], loc='lower left', fontsize=6)
    leg.get_frame().set_alpha(0.3)
    ax.set_title("CI", fontsize=9)

    # Reproduction Interval
    ax = axes[1]
    ax.scatter(X_train, y_train, color='grey', alpha=0.3, s=5)
    ax.plot(X_test_s, pred_s, '-', color=darken("#7CAE00",1.0), linewidth=1)
    ax.fill_between(X_test_s, ri_lb_s, ri_ub_s, facecolor="#7CAE00", edgecolor="#7CAE00", alpha=0.4)
    ax.plot(X_plot, true_plot, 'k--', linewidth=1, zorder=5, label='True Function')
    ax.text(0.05, 0.95, f'Coverage: {mean_ri_coverage:.2%}',
            transform=ax.transAxes, fontsize=8, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.2))
    leg = ax.legend(['Training Data', 'Prediction', 'RI', 'True Function'], loc='lower left', fontsize=6)
    leg.get_frame().set_alpha(0.3)
    ax.set_title("RI", fontsize=9)

    # Prediction Interval
    ax = axes[2]
    ax.scatter(X_train, y_train, color='grey', alpha=0.3, s=5)
    ax.plot(X_test_s, pred_s, '-', color=darken("#F8766D",1.0), linewidth=1)
    ax.fill_between(X_test_s, pi_lb_s, pi_ub_s, facecolor="#F8766D", edgecolor="#F8766D", alpha=0.4)
    ax.plot(X_plot, true_plot, 'k--', linewidth=1, zorder=5, label='True Function')
    ax.text(0.05, 0.95, f'Coverage: {mean_pi_coverage:.2%}',
            transform=ax.transAxes, fontsize=8, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.2))
    leg = ax.legend(['Training Data', 'Prediction', 'PI', 'True Function'], loc='lower left', fontsize=6)
    leg.get_frame().set_alpha(0.3)
    ax.set_title("PI", fontsize=9)

    # 4) Conformal Interval
    ax = axes[3]
    ax.scatter(X_train, y_train, color='grey', alpha=0.3, s=5)
    ax.plot(X_test_s, pred_s, '-', color=darken("#C77CFF",1.0), linewidth=1)
    ax.fill_between(X_test_s, cfml_lb_s, cfml_ub_s, facecolor="#C77CFF", edgecolor="#C77CFF", alpha=0.4)
    ax.plot(X_plot, true_plot, 'k--', linewidth=1, zorder=5, label='True Function')
    ax.text(0.05, 0.95, f'Coverage: {mean_cfml_coverage:.2%}',
            transform=ax.transAxes, fontsize=8, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.2))
    leg = ax.legend(['Training Data', 'Prediction', 'Conformal PI', 'True Function'], loc='lower left', fontsize=6)
    leg.get_frame().set_alpha(0.3)
    ax.set_title("Conformal PI", fontsize=9)

    plt.tight_layout()
    path = './1d/plots/'
    fig.savefig(path + f'ne_{ne}_lr_{lr}_md_{md}_sr_{sr}_dr_{dr}_here.png', dpi=500)
    plt.close(fig)

Grid Searching...:   0%|          | 0/1 [00:00<?, ?it/s]

Singular matrix encountered during sketching. Falling back to full K.


Iterating test points...: 300it [00:28, 10.70it/s]


Singular matrix encountered during sketching. Falling back to full K.


Iterating test points...: 300it [00:30,  9.87it/s]


...

Iterating test points...: 300it [00:17, 17.53it/s]


Singular matrix encountered during sketching. Falling back to full K.


Iterating test points...: 300it [00:16, 18.20it/s]


Singular matrix encountered during sketching. Falling back to full K.


Iterating test points...: 300it [00:20, 14.72it/s]
Grid Searching...: 100%|██████████| 1/1 [30:40<00:00, 1840.15s/it]
